# Import

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive, spark
from wmfdata.utils import print_err, pd_display_all

import warnings
warnings.filterwarnings('ignore')


# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")



In [3]:
datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

datetime.date(2023, 6, 30)

# Preparation

In [4]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    #"metrics_prev_month": str(metrics_month - 1),
    #"metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    #"metrics_month_end": str(last_month),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

In [5]:
date_params

{'mediawiki_history_snapshot': '2023-06',
 'metrics_month': '2023-06',
 'metrics_month_first_day': '2023-06-01',
 'metrics_month_last_day': '2023-06-30',
 'metrics_year': 2023,
 'metrics_cur_month': 6}

# MariaDB and Hive query metrics


In [6]:
# Code to Suppress PySpark Warning messages
from warnings import filterwarnings
filterwarnings("ignore")

In [7]:
queries = {
    "pageviews": {
        "file": "queries/pageviews.hql",
        "engine": "hive"
    },
    
    "automated_pageviews": {
        "file": "queries/automated_pageviews.hql",
        "engine": "hive"
    },
    "page_previews": {
        "file": "queries/page_previews.hql",
        "engine": "hive"
    },
    "unique_devices": {
        "file": "queries/unique_devices.hql",
        "engine": "hive"
    }


# Removing since global south metrics are calculated in notebook 02b-diversity-calculation.ipynb
   # ,
   # "global_south_pageviews": {
   #     "file": "queries/global_south_pageviews.hql",
   #     "engine": "hive"
   # },
    
   # "global_south_previews": {
   #     "file": "queries/global_south_previews.hql",
   #     "engine": "hive"
   # }
    
# Removing since mobile-heavy metrics are no longer tracked/reported on in the key product metrics presentations
    #"mobile-heavy_pageviews": {
    #    "file": "queries/mobile_heavy_pageviews.hql",
    #    "engine": "hive" }, 
    #"mobile-heavy_previews": {
    #   "file": "queries/mobile_heavy_previews.hql",
    #   "engine": "hive" }
    
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = spark.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result


SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Running pageviews on hive...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/07 22:27:06 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/07/07 22:27:06 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/07/07 22:27:06 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
23/07/07 22:27:06 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
23/07/07 22:27:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/07 22:27:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/07/07 22:27:07 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/07/07 22:27:09 WARN Client: E

Running page_previews on hive...                                                


Running unique_devices on hive...                                               


# Combining and saving metrics

In [8]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

# Add Metrics for Interactions
new_metrics['interactions'] = new_metrics.apply(lambda x: x['previews_seen'] + x['total_pageview'], axis=1)


In [9]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

,automated_pageviews,desktop,interactions,mobileweb,previews_seen,total_pageview,unique_devices
month,,,,,,,
2022-09-01,2.705377e+09,6.129059e+09,1.986889e+10,1.175190e+10,1.706285e+09,1.816260e+10,1.554616e+09
2022-10-01,2.722851e+09,6.677566e+09,2.049623e+10,1.178869e+10,1.740051e+09,1.875618e+10,1.683358e+09
2022-11-01,3.125762e+09,6.639885e+09,2.103694e+10,1.239146e+10,1.725951e+09,1.931098e+10,1.602483e+09
2022-12-01,1.514616e+09,6.245363e+09,1.853431e+10,1.033699e+10,1.645032e+09,1.688927e+10,1.580337e+09
2023-01-01,2.058892e+09,6.655879e+09,2.052910e+10,1.169615e+10,1.853753e+09,1.867535e+10,1.623196e+09
2023-02-01,2.484894e+09,5.809895e+09,1.896141e+10,1.123269e+10,1.643540e+09,1.731786e+10,1.556624e+09
2023-03-01,2.851128e+09,6.554146e+09,2.110827e+10,1.244635e+10,1.808851e+09,1.929942e+10,1.666366e+09
2023-04-01,1.835848e+09,5.903055e+09,1.857836e+10,1.074450e+10,1.635800e+09,1.694256e+10,1.653133e+09
2023-05-01,2.788857e+09,6.136982e+09,2.086826e+10,1.269368e+10,1.738922e+09,1.912934e+10,1.596438e+09


In [10]:
metrics.to_csv(FILENAME, sep="\t")